In [1]:
import concurrent.futures
import collections
import math
import os
import pathlib
import random
import re
import string
import time

import einops
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import tqdm

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_datasets as tfds

In [2]:
def flickr8k():
    path = pathlib.Path.cwd()
    
    if len(list(path.rglob('*'))) < 16197:
        tf.keras.utils.get_file(
            origin='https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip',
            cache_dir=path,
            cache_subdir='datasets',
            extract=True)
        tf.keras.utils.get_file(
            origin='https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip',
            cache_dir=path,
            cache_subdir='datasets',
            extract=True)
    
    captions = (path/"Flickr8k.token.txt").read_text().splitlines()
    captions = (line.split('\t') for line in captions)
    captions = ((fname.split('#')[0], caption) for (fname, caption) in captions)
    
    cap_dict = collections.defaultdict(list)
    for fname, cap in captions:
        cap_dict[fname].append(cap)

    path = str(path) + '\\datasets'
    
    train_files = (path/'Flickr_8k.trainImages.txt').read_text().splitlines()
    train_captions = [(str(path/'Flicker8k.Dataset'/fname), cap_dict[fname]) for fname in train_files]
    
    test_files = (path/'Flickr_8k.testImages.txt').read_text().splitlines()
    test_captions = [(str(path/'Flicker8k.Dataset'/fname), cap_dict[fname]) for fname in test_files]
    
    train_ds = tf.data.experimental.from_list(train_captions)
    test_ds = tf.data.experimental.from_list(test_captions)
    
    return train_ds, test_ds

In [ ]:
train_raw, test_raw = flickr8k()

 932585472/1115419746 [========================>.....] - ETA: 1:58

In [ ]:
for ex_path, ex_captions in train_raw.take(1):
    print(ex_path)
    print(ex_captions)